![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [ ]:
# Importer QuantBook et les bibliothèques nécessaires
from AlgorithmImports import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import seaborn as sns

# Initialiser QuantBook
qb = QuantBook()

# Définir le sous-jacent
symbol = qb.AddEquity("SPY").Symbol

# Charger l'historique des prix
start_date = datetime(2020, 6, 1)
end_date = datetime(2024, 6, 1)
history = qb.History(symbol, start_date, end_date, Resolution.Daily)

# Extraire les données pour le symbole SPY avec un index simple (temps uniquement)
symbol_history = history.xs(symbol, level='symbol')
print("Aperçu des données symbol_history :")
print(symbol_history.head())

# Récupérer la liste des contrats d'options disponibles
contracts = qb.OptionChainProvider.GetOptionContractList(symbol, datetime.now())
print(f"Nombre total de contrats disponibles : {len(contracts)}")

# Filtrer les expirations disponibles
expirations = sorted(set([c.ID.Date.date() for c in contracts]))
if not expirations:
    print("Aucune expiration disponible pour les contrats d'options. Fin de l'analyse.")
else:
    print(f"Expirations disponibles (TOP 5): {expirations[:5]}")

    # Définir une expiration cible
    target_expiry = expirations[0]

    # Séparer PUTs et CALLs pour l'expiration cible
    puts = [c for c in contracts if c.ID.Date.date() == target_expiry and c.ID.OptionRight == OptionRight.PUT]
    calls = [c for c in contracts if c.ID.Date.date() == target_expiry and c.ID.OptionRight == OptionRight.CALL]

    print(f"PUTs disponibles pour expiration {target_expiry}:")
    for p in puts[:5]:
        print(f"Strike: {p.ID.StrikePrice}, Expiry: {p.ID.Date}")

    print(f"CALLs disponibles pour expiration {target_expiry}:")
    for c in calls[:5]:
        print(f"Strike: {c.ID.StrikePrice}, Expiry: {c.ID.Date}")

    # Distribution des strikes
    plt.figure(figsize=(10, 6))
    sns.histplot([p.ID.StrikePrice for p in puts], bins=20, label="PUTs", color="blue", alpha=0.6)
    sns.histplot([c.ID.StrikePrice for c in calls], bins=20, label="CALLs", color="red", alpha=0.6)
    plt.title("Distribution des Strikes pour PUTs et CALLs")
    plt.xlabel("Strike Price")
    plt.ylabel("Nombre de contrats")
    plt.legend()
    plt.show()

    # Calcul des primes (approximatives)
    strike_prices = [p.ID.StrikePrice for p in puts]
    underlying_price = symbol_history['close'].iloc[-1]  # Dernier prix connu
    put_premiums = [max(0, underlying_price - sp) for sp in strike_prices]
    call_premiums = [max(0, sp - underlying_price) for sp in strike_prices]

    plt.figure(figsize=(10, 6))
    plt.plot(strike_prices, put_premiums, label="PUT Premiums", marker='o', color="blue")
    plt.plot(strike_prices, call_premiums, label="CALL Premiums", marker='x', color="red")
    plt.title("Prime estimée pour PUTs et CALLs")
    plt.xlabel("Strike Price")
    plt.ylabel("Premium (USD)")
    plt.legend()
    plt.grid()
    plt.show()

    # Simulation des signaux (stratégie Wheel)
    otm_threshold = 0.05  # 5% OTM
    simulated_puts = []
    simulated_calls = []

    for date in symbol_history.index:
        price = symbol_history.loc[date, 'close']

        # PUT signal
        put_strike = price * (1 - otm_threshold)
        put_contracts = [p for p in puts if p.ID.StrikePrice <= put_strike]
        if put_contracts:
            best_put = sorted(put_contracts, key=lambda x: x.ID.StrikePrice, reverse=True)[0]
            simulated_puts.append((date, best_put.ID.StrikePrice))

        # CALL signal (après assignation PUT)
        call_strike = price * (1 + otm_threshold)
        call_contracts = [c for c in calls if c.ID.StrikePrice >= call_strike]
        if call_contracts:
            best_call = sorted(call_contracts, key=lambda x: x.ID.StrikePrice, reverse=False)[0]
            simulated_calls.append((date, best_call.ID.StrikePrice))

    # Visualisation des signaux simulés
    plt.figure(figsize=(14, 8))
    plt.plot(symbol_history.index, symbol_history['close'], label="SPY Price", color="black", linewidth=1.5)

    if simulated_puts:
        put_dates, put_strikes = zip(*simulated_puts)
        plt.scatter(put_dates, put_strikes, marker='^', color='green', label="PUT Sold")

    if simulated_calls:
        call_dates, call_strikes = zip(*simulated_calls)
        plt.scatter(call_dates, call_strikes, marker='v', color='red', label="CALL Sold")

    plt.title("SPY Price and Wheel Strategy Signals")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.grid()
    plt.show()

    # Résumé des paramètres
    print(f"Simulation réalisée avec les paramètres :")
    print(f" - Seuil OTM (put et call): {otm_threshold * 100}%")
    print(f" - Expiration cible : {target_expiry}")
